In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("whitegrid")

from sklearn.preprocessing import OrdinalEncoder

from datgan import stats_assessment
from datgan import ml_assessment, transform_results


# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
df_orig = pd.read_csv('../../data/LPMC/trips.csv', index_col=False)

continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt','dur_driving', 'driving_traffic_percent']

cond_inputs = ["age", "female", "hh_borough"]

synth_files = []

for i in range(5):
    synth_files.append('../../data/synthetic/test/ciDATGAN_{:02d}.csv'.format(i+1))
    synth_files.append('../../data/synthetic/test/DATGAN_{:02d}.csv'.format(i+1))

len_df = len(df_orig)

In [3]:
results_path = './results/'

if not os.path.exists(results_path):
    os.makedirs(results_path)

In [4]:
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']
colors = [(0.6627450980392157, 0.6627450980392157, 0.6627450980392157, 1.0), (1.0, 0.0, 0.0, 0.0)]

# First level

In [5]:
pickle_name = 'res_lvl1.pickle'
aggregation_level = 1

first_lvl_stats = {}

try:
    first_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickle file, using that')
except:
    print('No previous results found, starting fresh')

No previous results found, starting fresh


In [6]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in first_lvl_stats:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        first_lvl_stats[file_name] = {}

        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        first_lvl_stats[file_name] = stats

    pickle.dump(first_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Preparing stats for file ciDATGAN_01 (1/10)
Preparing stats for file DATGAN_01 (2/10)
Preparing stats for file ciDATGAN_02 (3/10)
Preparing stats for file DATGAN_02 (4/10)
Preparing stats for file ciDATGAN_03 (5/10)
Preparing stats for file DATGAN_03 (6/10)
Preparing stats for file ciDATGAN_04 (7/10)
Preparing stats for file DATGAN_04 (8/10)
Preparing stats for file ciDATGAN_05 (9/10)
Preparing stats for file DATGAN_05 (10/10)
FINISHED!


In [7]:
res = {}

for test in ['all', 'cont', 'cat']:

    res[test] = {}

    if test == 'all':
        cols = df_orig.columns
    elif test == 'cont':
        cols = continuous_columns
    elif test == 'cat':
        cols = set(df_orig.columns) - set(continuous_columns)

    cols = set(cols) - set(cond_inputs)

    for s in stats_str:
        res[test][s] = {}

    for m in first_lvl_stats.keys():
        for s in stats_str:
            res[test][s][m] = []

            for c in cols:
                res[test][s][m].append(first_lvl_stats[m][c][s])

In [8]:
sts = 'srmse'

res_DATGAN = []
res_ciDATGAN = []

for i in range(5):
    res_DATGAN.append(res['all'][sts]['DATGAN_{:02d}'.format(i+1)])
    res_ciDATGAN.append(res['all'][sts]['ciDATGAN_{:02d}'.format(i+1)])

res_DATGAN = np.array(res_DATGAN).flatten()
res_ciDATGAN = np.array(res_ciDATGAN).flatten()

tmp = [res_DATGAN, res_ciDATGAN]
df = pd.DataFrame(tmp, index=['DATGAN', 'ciDATGAN'])

plt.figure(figsize=(10,7))

sns.violinplot(data=df.T, palette=colors)

plt.xticks([0,1], ['DATGAN', 'ciDATGAN'])
plt.ylabel(sts.upper())

plt.savefig('../../figures/obs/lvl1.png'.format(s), bbox_inches='tight')
plt.savefig('../../figures/obs/lvl1.pdf'.format(s), bbox_inches='tight')
plt.close()

# Second level

In [9]:
pickle_name = 'res_lvl2.pickle'
aggregation_level = 2

second_lvl_stats = {}

try:
    second_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

No previous results found, starting fresh


In [10]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in second_lvl_stats:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        second_lvl_stats[file_name] = {}

        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        second_lvl_stats[file_name] = stats

    pickle.dump(second_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Preparing stats for file ciDATGAN_01 (1/10)
Preparing stats for file DATGAN_01 (2/10)
Preparing stats for file ciDATGAN_02 (3/10)
Preparing stats for file DATGAN_02 (4/10)
Preparing stats for file ciDATGAN_03 (5/10)
Preparing stats for file DATGAN_03 (6/10)
Preparing stats for file ciDATGAN_04 (7/10)
Preparing stats for file DATGAN_04 (8/10)
Preparing stats for file ciDATGAN_05 (9/10)
Preparing stats for file DATGAN_05 (10/10)
FINISHED!


In [11]:
res = {}

for s in stats_str:
    res[s] = {}

for m in second_lvl_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for c in second_lvl_stats[m].keys():
            res[s][m].append(second_lvl_stats[m][c][s])

In [12]:
sts = 'srmse'

res_DATGAN = []
res_ciDATGAN = []

for i in range(5):
    res_DATGAN.append(res[sts]['DATGAN_{:02d}'.format(i+1)])
    res_ciDATGAN.append(res[sts]['ciDATGAN_{:02d}'.format(i+1)])

res_DATGAN = np.array(res_DATGAN).flatten()
res_ciDATGAN = np.array(res_ciDATGAN).flatten()

tmp = [res_DATGAN, res_ciDATGAN]
df = pd.DataFrame(tmp, index=['DATGAN', 'ciDATGAN'])

plt.figure(figsize=(10,7))

sns.violinplot(data=df.T, palette=colors)

plt.xticks([0,1], ['DATGAN', 'ciDATGAN'])
plt.ylabel(sts.upper())

plt.savefig('../../figures/obs/lvl2.png'.format(s), bbox_inches='tight')
plt.savefig('../../figures/obs/lvl2.pdf'.format(s), bbox_inches='tight')
plt.close()

# Machine Learning efficacy

In [13]:
def check_low_appearing_vars(df):

    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print('Variable {}: '.format(c))
                for idx, v in zip(val.index, val):
                    if v < 0.01:
                        print('  {} - {:.2f}% ({:d})'.format(idx, 100*v, int(v*len(df))))
                print()

def replace_low_appearing_values(df):

    dct_ = {}
    for i in df['hh_vehicles'].unique():
        if i >= 3:
            dct_[i] = '3+'
        else:
            dct_[i] = str(i)
    df['hh_vehicles'].replace(dct_, inplace=True)

    dct_ = {}
    for i in df['hh_people'].unique():
        if i >= 6:
            dct_[i] = '6+'
        else:
            dct_[i] = str(i)
    df['hh_people'].replace(dct_, inplace=True)

In [14]:
check_low_appearing_vars(df_orig)

Variable hh_vehicles: 
  4 - 0.52% (88)
  5 - 0.12% (21)
  6 - 0.02% (3)
  8 - 0.02% (3)
  7 - 0.01% (2)

Variable hh_people: 
  7 - 0.70% (118)
  8 - 0.18% (31)
  9 - 0.08% (13)
  10 - 0.03% (5)
  11 - 0.02% (3)



In [15]:
replace_low_appearing_values(df_orig)

In [16]:
check_low_appearing_vars(df_orig)

In [17]:
categorical_columns = list(set(df_orig.columns) - set(continuous_columns))

In [18]:
enc = OrdinalEncoder()
df_orig[categorical_columns] = enc.fit_transform(df_orig[categorical_columns])

In [19]:
pickle_name = 'ml.pickle'

cv_modelscores = {}

try:
    cv_modelscores = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

No previous results found, starting fresh


In [20]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in cv_modelscores:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        cv_modelscores[file_name] = {}

        # Load the synthetic dataset
        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        # Replace the values rarely appearing
        replace_low_appearing_values(df_synth)

        # Encode the synthetic dataset
        df_synth[categorical_columns] = enc.transform(df_synth[categorical_columns])

        res = ml_assessment(df_orig, df_synth, continuous_columns, categorical_columns, ignore_cols=cond_inputs)

        cv_modelscores[file_name] = res

    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Preparing stats for file ciDATGAN_01 (1/10)
Preparing stats for file DATGAN_01 (2/10)
Preparing stats for file ciDATGAN_02 (3/10)
Preparing stats for file DATGAN_02 (4/10)
Preparing stats for file ciDATGAN_03 (5/10)
Preparing stats for file DATGAN_03 (6/10)
Preparing stats for file ciDATGAN_04 (7/10)
Preparing stats for file DATGAN_04 (8/10)
Preparing stats for file ciDATGAN_05 (9/10)
Preparing stats for file DATGAN_05 (10/10)
FINISHED!                         


In [21]:
if 'original' in cv_modelscores:
    print("Results for file \033[1m{}\033[0m already exists!".format('original'))
else:
    print("Preparing stats for file \033[1m{}\033[0m".format('original'))

    res = ml_assessment(df_orig, df_orig, continuous_columns, categorical_columns, ignore_cols=cond_inputs)
    cv_modelscores['original'] = res
    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))
    print("\033[1mFINISHED!\033[0m")

Preparing stats for file original
FINISHED!                         


In [22]:
res_ml = {
    'DATGAN': {'continuous': [], 'categorical': []},
    'ciDATGAN': {'continuous': [], 'categorical': []}
}

continuous_columns = list(set(continuous_columns) - set(cond_inputs))
categorical_columns = list(set(categorical_columns) - set(cond_inputs))

ori_scores = {col: cv_modelscores['original'][col]['test_log_loss'] for col in categorical_columns}
ori_scores.update({col: cv_modelscores['original'][col]['test_l2'] for col in continuous_columns})

for i in range(5):

    for f in ['DATGAN', 'ciDATGAN']:

        name = '{}_{:02d}'.format(f, i+1)

        external = {}

        for col in categorical_columns:
            res_ml[f]['categorical'].append(cv_modelscores[name][col]['original_log_loss'] - ori_scores[col])

        for col in continuous_columns:
            res_ml[f]['continuous'].append(cv_modelscores[name][col]['original_l2']/ori_scores[col])

In [23]:
for c in ['continuous', 'categorical']:

    tmp = [res_ml['DATGAN'][c], res_ml['ciDATGAN'][c]]
    df = pd.DataFrame(tmp, index=['DATGAN', 'ciDATGAN'])

    plt.figure(figsize=(10,7))

    sns.violinplot(data=df.T, palette=colors)

    plt.xticks([0,1], ['DATGAN', 'ciDATGAN'])
    if c == 'continuous':
        plt.ylabel('Relative L2 error')
    else:
        plt.ylabel('Relative log loss')

    plt.savefig('../../figures/obs/ml_{}.png'.format(c[:3]), bbox_inches='tight')
    plt.savefig('../../figures/obs/ml_{}.pdf'.format(c[:3]), bbox_inches='tight')
    plt.close()